## Day 22 - 3D sand block simulation... :panik:

In [17]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['1,0,1~1,2,1',
 '0,0,2~2,0,2',
 '0,2,3~2,2,3',
 '0,0,4~0,2,4',
 '2,0,5~2,2,5',
 '0,1,6~2,1,6',
 '1,1,8~1,1,9']

Each row describes the coordinates of a block, starting from lowest z coord to highest.
- format is x1,y1,z1~x2,y2,z2 to describe the volume that block spans

Blocks are supported if there's at least one contact point below, otherwise they fall.

**Part 1: Simulate such that all the blocks settle (some haven't 'landed' down yet) and then find out how many blocks can be removed without things falling down!**

In [18]:
from dataclasses import dataclass

@dataclass
class Brick():
    x: tuple[int, int]
    y: tuple[int, int]
    z: tuple[int, int]

    # def can_balance_on_this(self, other_brick: 'Brick'):
    #     z1s, z1e = self.z
    #     z2s, z2e = other_brick.z

    #     return z2s in range(z1s, z1e) or z2e in range(z1s, z1e)

    def fall_vertically_to_row(self, row: int):
        z1, z2 = self.z
        assert row < z1, f"{row=} {z1=}"
        diff = z1 - row
        self.z = (z1-diff, z2-diff)
    
    # def overlap_horizontally(self, other_brick: 'Brick'):
    #     return (
    #         (
    #         other_brick.x[0] in range(self.x[0], self.x[1]+1) or
    #         other_brick.x[1] in range(self.x[0], self.x[1]+1) or
    #         self.x[0] in range(other_brick.x[0], other_brick.x[1]+1) or
    #         self.x[1] in range(other_brick.x[0], other_brick.x[1]+1)
    #         ) and
    #         (
    #         other_brick.y[0] in range(self.y[0], self.y[1]+1) or
    #         other_brick.y[1] in range(self.y[0], self.y[1]+1) or
    #         self.y[0] in range(other_brick.y[0], other_brick.y[1]+1) or
    #         self.y[1] in range(other_brick.y[0], other_brick.y[1]+1)
    #         )
    #     )
    def overlap_horizontally(self, other_brick: 'Brick'):
        return (
            max(other_brick.x[0], self.x[0]) <= min(other_brick.x[1], self.x[1])
            and max(other_brick.y[0], self.y[0]) <= min(other_brick.y[1], self.y[1])
        )

In [19]:
brick1 =Brick(x=(2, 3), y=(8, 8), z=(1, 1))
brick2 =Brick(x=(1, 3), y=(7, 7), z=(1, 1))

brick1.overlap_horizontally(brick2)

False

In [20]:
class Stack():

    def __init__(self, bricks: list[Brick]):
        self.bricks: list[Brick] = bricks
        bricks.sort(key=lambda b: b.z[0])
        self.shift_bricks_down_as_far_as_they_can()
        bricks.sort(key=lambda b: b.z[0])

    
    # def shift_bricks_down_as_far_as_they_can(self):
    #     z_max = max([brick.z[1] for brick in self.bricks])

    #     def set_flat_rows() -> dict[int, list[Brick]]:
    #         flat_rows = {z:[] for z in range(1, z_max+1)}
    #         for brick in self.bricks:
    #             for z in range(brick.z[0], brick.z[1]+1):
    #                 flat_rows[z].append(brick)
    #         return flat_rows
        
    #     flat_rows = set_flat_rows()

    #     for brick in self.bricks:
    #         for z in range(brick.z[0]-1, 0, -1):
    #             doesnt_overlap = True
    #             for sub_brick in flat_rows[z]:
    #                 if brick != sub_brick:
    #                     doesnt_overlap = not sub_brick.overlap_horizontally(brick)
    #             if doesnt_overlap:
    #                 brick.fall_vertically_to_row(z)
    #                 flat_rows = set_flat_rows()
    #             else:
    #                 # must be an overlap
    #                 break

    def shift_bricks_down_as_far_as_they_can(self):
        for idx, brick in enumerate(self.bricks):
            max_z = 1
            for check in self.bricks[:idx]:
                if brick.overlap_horizontally(check):
                    max_z = max(max_z, check.z[1] + 1)
            brick.z = (max_z, brick.z[1] - (brick.z[0] - max_z))


In [21]:
# SET UP (slow to run so get it defined etc.)

# assert False, "Put it here to avoid overwriting input_stack lol. Can't be waiting another couple minutes!"

bricks = []
for line in input_lines:
    l, r = line.split("~")
    lx, ly, lz = map(int, l.split(","))
    rx, ry, rz = map(int, r.split(","))
    bricks.append(
        Brick((lx, rx), (ly, ry), (lz, rz))
    )
input_stack = Stack(bricks)

In [22]:
for brick in input_stack.bricks:
    print(brick)

Brick(x=(6, 8), y=(4, 4), z=(1, 1))
Brick(x=(1, 3), y=(7, 7), z=(1, 1))
Brick(x=(2, 3), y=(8, 8), z=(1, 1))
Brick(x=(6, 6), y=(1, 3), z=(1, 1))
Brick(x=(4, 5), y=(3, 3), z=(1, 1))
Brick(x=(1, 1), y=(1, 1), z=(1, 3))
Brick(x=(0, 0), y=(3, 3), z=(1, 2))
Brick(x=(0, 3), y=(9, 9), z=(1, 1))
Brick(x=(4, 4), y=(6, 9), z=(1, 1))
Brick(x=(1, 2), y=(2, 2), z=(1, 1))
Brick(x=(7, 9), y=(3, 3), z=(1, 1))
Brick(x=(3, 3), y=(0, 1), z=(1, 1))
Brick(x=(9, 9), y=(7, 7), z=(1, 4))
Brick(x=(7, 9), y=(0, 0), z=(1, 1))
Brick(x=(4, 5), y=(5, 5), z=(1, 1))
Brick(x=(1, 1), y=(6, 6), z=(1, 4))
Brick(x=(3, 3), y=(4, 6), z=(1, 1))
Brick(x=(6, 6), y=(5, 7), z=(1, 1))
Brick(x=(3, 3), y=(2, 3), z=(1, 1))
Brick(x=(2, 2), y=(5, 7), z=(2, 2))
Brick(x=(6, 6), y=(0, 1), z=(2, 2))
Brick(x=(0, 1), y=(2, 2), z=(2, 2))
Brick(x=(5, 7), y=(7, 7), z=(2, 2))
Brick(x=(7, 7), y=(4, 4), z=(2, 3))
Brick(x=(3, 3), y=(7, 7), z=(2, 4))
Brick(x=(3, 3), y=(0, 0), z=(2, 4))
Brick(x=(7, 7), y=(0, 0), z=(2, 3))
Brick(x=(0, 0), y=(7, 9), z=

In [23]:
def part1(lines: list[str]):
    bricks = []
    for line in lines:
        l, r = line.split("~")
        lx, ly, lz = map(int, l.split(","))
        rx, ry, rz = map(int, r.split(","))
        bricks.append(
            Brick((lx, rx), (ly, ry), (lz, rz))
        )
    stack = Stack(bricks)

    k_supports_v = {i: set() for i in range(len(stack.bricks))}
    v_supports_k = {i: set() for i in range(len(stack.bricks))}

    for j, upper in enumerate(stack.bricks):
        for i, lower in enumerate(stack.bricks[:j]):
            if upper.overlap_horizontally(lower) and upper.z[0] == lower.z[1] + 1:
                k_supports_v[i].add(j)
                v_supports_k[j].add(i)
                
    total = 0
    for i in range(len(stack.bricks)):
        if all(len(v_supports_k[j]) >= 2 for j in k_supports_v[i]):
            total += 1
    
    return total               
                
    

assert part1(example_lines)
part1(input_lines) # on real input, 965 is TOO HIGH


432

**Part 2 - Chaos, how many bricks can we make fall?**

In [31]:
from collections import deque

def part2(lines: list[str]):
    bricks = []
    for line in lines:
        l, r = line.split("~")
        lx, ly, lz = map(int, l.split(","))
        rx, ry, rz = map(int, r.split(","))
        bricks.append(
            Brick((lx, rx), (ly, ry), (lz, rz))
        )
    stack = Stack(bricks)

    k_supported_by_v = {i: set() for i in range(len(stack.bricks))}
    v_supports_k = {i: set() for i in range(len(stack.bricks))}

    for j, upper in enumerate(stack.bricks):
        for i, lower in enumerate(stack.bricks[:j]):
            if upper.overlap_horizontally(lower) and upper.z[0] == lower.z[1] + 1:
                k_supported_by_v[i].add(j)
                v_supports_k[j].add(i)

    # v_supports_k is lowers supporting uppers
                
    total = 0
    for i in range(len(stack.bricks)):
        # things about to fall is q
        q = deque(j for j in k_supported_by_v[i] if len(v_supports_k[j]) == 1)  # add to deque everything caused to fall (i.e. those that were supported solely by i)
        falling = set(q)
        falling.add(i)
        while q:
            j = q.popleft()
            for k in k_supported_by_v[j] - falling:
                # everything supported by j that isn't falling!
                if v_supports_k[k] <= falling: # <= means subset here
                    q.append(k)
                    falling.add(k)
        
        total += len(falling) - 1
    
    return total

assert part2(example_lines) == 7
part2(input_lines)

63166